In [1]:
import pandas as pd
import os
from datetime import datetime, date
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing

import warnings
warnings.filterwarnings('ignore')

# This function converts given date to age
def age(born, days_present = True):
    today = date.today()
    if not days_present:
        return today.year - born.year - int(today.month < born.month)    
    return today.year - born.year - ((today.month, today.day) <
                                     (born.month, born.day))

## Import and preprocess OPENonOH demographics and statistical data
Get it into the same shape as the OpenAPS dataset (OpenAPS complete_patient_statistics.xlsx)

In [2]:
dir_ = "/home/reinhold/Daten/Paper_Datasets_Nov2022"
filenames = [
    r'FINAL n=75 statistics processed.xlsx', r'Demo+SRCO_Adults_Caregivers.xlsx', r'n=75_deduped_demographics.xlsx'] # Specify the file path

OpenAPS_columns = ["id","interdaysd", "interdaycv","bg_roc_stds","TOR<70","TIR","TOR>180","POR","J_index","LGBI","HBGI","GMI","count","mean+std","mean-std","mean","std",
    "min","25 %","50 %","75 %","max","age","dailyInsulinUnits","dailyBasalInsulinUnits","height","weight","gender","country","DIYTech"]


adapt the demographics file's variable naming

In [3]:

#columns_ = ["age","height", "weight", "TOR<70", "TIR", "TOR>180"] # "dailyInsulinUnits", "dailyBasalInsulinUnits", "dailyCarbs" missing
col_maps = {"TBR<70 [%]" : "TOR<70", "TIR [%]" : "TIR", "TAR>180 [%]" : "TOR>180", "year_of_birth" : "year", "month_of_birth" : "month", "country_of_origin" : "country"}

df = pd.read_excel(os.path.join(dir_, filenames[1]), sheet_name="n=75_deduped_compressed")
#df_OPENonOH_2 = pd.read_excel(os.path.join(dir_, filenames[2]))

df.rename(columns=col_maps, inplace=True)  # to enable the age()-function defined above
print([c for c in df.columns if "[\%]" in c])
df['age'] = df.apply(lambda x: age(x, False), axis=1)
df['height'] = df['height_cm_v2']
df.loc[df['height'].isnull(), 'height'] = df.loc[df['height'].isnull(), 'height_inches_v2'] * 30.48 + df.loc[df['height'].isnull(), 'height_inches_2_v2'] * 2.54
df['weight'] = df['body_weight_kg_v2']
df.loc[df['weight'].isnull(), 'weight'] = df.loc[df['weight'].isnull(), 'body_weight_pounds_v2'] * 0.453592

df = df[["age", "project_member_id", "gender", "height", "weight", "country", "year", "month"]]


[]


read the blood glucose data file and merge it with the demographics information

In [4]:

df_BG_stats = pd.read_excel(os.path.join(dir_, filenames[0]))
df_out = pd.merge(df, df_BG_stats, left_on='project_member_id', right_on='id', how='inner')

#df.drop(columns=['id', "height_cm_v2", "month", "year"], inplace=True)
df_out.drop(columns=['project_member_id'], inplace=True)
new_col_sequence = ["id"]
new_col_sequence.extend([c for c in df_out.columns if c !="id"])
df_out = df_out.reindex(columns = new_col_sequence)
df_out.to_excel(os.path.join(dir_, "OPENonOH complete_patient_statistics_test.xlsx"), index=False)
print(f'saved to: {os.path.join(dir_, "OPENonOH complete_patient_statistics_test.xlsx")}')

saved to: /home/reinhold/Daten/Paper_Datasets_Nov2022/OPENonOH complete_patient_statistics_test.xlsx


In [5]:
df_out.info()
len(df.columns), df.columns
df_out.head(2)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 0 to 74
Data columns (total 27 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           75 non-null     int64  
 1   age          66 non-null     float64
 2   gender       67 non-null     float64
 3   height       66 non-null     float64
 4   weight       65 non-null     float64
 5   country      67 non-null     float64
 6   year         66 non-null     float64
 7   month        66 non-null     float64
 8   interdaysd   75 non-null     float64
 9   interdaycv   75 non-null     float64
 10  bg_roc_stds  75 non-null     float64
 11  TBR<70 [%]   75 non-null     float64
 12  TIR [%]      75 non-null     float64
 13  TAR>180 [%]  75 non-null     float64
 14  POR          75 non-null     float64
 15  J_index      75 non-null     float64
 16  LGBI         75 non-null     float64
 17  HBGI         75 non-null     float64
 18  GMI          75 non-null     float64
 19  count     

,id,age,gender,height,weight,country,year,month,interdaysd,interdaycv,...,HBGI,GMI,count,mean,std,min,0.25,0.5,0.75,max
0,27718918,NaN,1.0,180.00,80.0,82.0,NaN,NaN,52.012219,38.276739,...,4.036825,6.560361,120437,135.884665,52.012434,40,96,123,166,400
1,37159654,28.0,1.0,157.48,75.0,229.0,1994.0,11.0,75.950309,50.233776,...,7.252564,6.926553,25745,151.193707,75.951784,40,95,131,188,400
